<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/colab_d_tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tunes as input

In [2]:
!pip uninstall gensim -y

Found existing installation: gensim 4.1.2
Uninstalling gensim-4.1.2:
  Successfully uninstalled gensim-4.1.2


In [3]:
!pip install gensim

  Using cached gensim-4.1.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)


In [4]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [77]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Simplified chords, with duplicates

In [78]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy' -O input_seq.txt


--2021-11-18 20:50:48--  https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy
Resolving docs.google.com (docs.google.com)... 209.85.200.101, 209.85.200.102, 209.85.200.113, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nsoeuntj4dfnm6v9ga3u6hfvuei45t7u/1637268600000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download [following]
--2021-11-18 20:50:48--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nsoeuntj4dfnm6v9ga3u6hfvuei45t7u/1637268600000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-10-4c-docs.googleusercontent.com (doc-10-4c-

### Major Triads, Minor Triads and Dom7 chords:

In [79]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-18 20:50:49--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 209.85.200.138, 209.85.200.102, 209.85.200.139, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4ds7ipetmo5fgdsksbvr0uaebbstvj0s/1637268600000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-18 20:50:49--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/4ds7ipetmo5fgdsksbvr0uaebbstvj0s/1637268600000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08-4c-

### Simplified chords, no repetitions

In [80]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=18aqk_RtyCssrtFljjGo9icwgTYXcmKck' -O input_seq.txt


--2021-11-18 20:50:49--  https://docs.google.com/uc?export=download&id=18aqk_RtyCssrtFljjGo9icwgTYXcmKck
Resolving docs.google.com (docs.google.com)... 209.85.200.139, 209.85.200.138, 209.85.200.100, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/da9q5s5lk9dnu8n7j5tqf2etl6t8gfe1/1637268600000/14329102864480165501/*/18aqk_RtyCssrtFljjGo9icwgTYXcmKck?e=download [following]
--2021-11-18 20:50:50--  https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/da9q5s5lk9dnu8n7j5tqf2etl6t8gfe1/1637268600000/14329102864480165501/*/18aqk_RtyCssrtFljjGo9icwgTYXcmKck?e=download
Resolving doc-0k-4c-docs.googleusercontent.com (doc-0k-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-0k-4c-docs.googleusercontent.com (doc-0k-4c-

### Basic Chords, no repetitions

In [81]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe' -O input_seq.txt


--2021-11-18 20:50:50--  https://docs.google.com/uc?export=download&id=1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe
Resolving docs.google.com (docs.google.com)... 74.125.124.138, 74.125.124.102, 74.125.124.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.124.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5nonfccjtt03r21q96opricm2dd7p6b7/1637268600000/14329102864480165501/*/1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe?e=download [following]
--2021-11-18 20:50:51--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/5nonfccjtt03r21q96opricm2dd7p6b7/1637268600000/14329102864480165501/*/1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-

### Meta Data

In [82]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-18 20:50:51--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 108.177.111.113, 108.177.111.101, 108.177.111.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.111.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7aj70i7uti4d4lm64g4fn5632fp4mr1r/1637268600000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-18 20:50:52--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7aj70i7uti4d4lm64g4fn5632fp4mr1r/1637268600000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14

### Read Data

In [83]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [84]:
data = [line.split(' ') for line in lines]

### Meta Data

In [85]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [86]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [87]:
assert len(data) == len(titles)
print(len(data))

1812


In [88]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [89]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [90]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [91]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [92]:
data_ngrams = list_to_ngrams(data, 3)

In [93]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [94]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3461
109,A7-Dm7-G7,1558
67,D7-G7-C,1164
34,Em7-A7-Dm7,1153
62,C-G7-C,859
155,A7-D7-G7,782
59,Am7-Dm7-G7,675
61,G7-C-G7,645
64,C-C7-F,639
6,Gm7-C7-F,625


In [95]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

# List Tunes for Test Data

In [96]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [97]:
contrafacts_test[:5]

[[0, 251], [3, 1527], [30, 63], [80, 1527], [104, 1372]]

# TF-IDF

## Build Model

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [99]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
tfidf_vectorizer = {}

for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  print(data_ngrams[0])
  data_tfidf = [" ".join(i) for i in data_ngrams]

  # make sure that no processing on the chord symbols is done!
  tv = TfidfVectorizer(min_df = 0., 
                     max_df = 1., 
                     norm = 'l2', 
                     use_idf = True,
                     stop_words=None,
                     lowercase=False,
                     token_pattern='\\b[\\w#]+')

  tfidf_vectorizer[n] = tv
  tfidf_model[n] = tv.fit_transform(data_tfidf)

['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C']
['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C']
['C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-F', 'C7-F-G#7', 'F-G#7-C#', 'G#7-C#-E7', 'C#-E7-

In [100]:
for i in range(len(tfidf_vectorizer)):
  print(tfidf_vectorizer[i+1].get_feature_names_out())

['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F7' 'Fm' 'Fm7'
 'G' 'G#' 'G#7' 'G#m' 'G#m7' 'G7' 'Gm' 'Gm7']
['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F7' 'Fm' 'Fm7'
 'G' 'G#' 'G#7' 'G#m' 'G#m7' 'G7' 'Gm' 'Gm7']
['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F7' 'Fm' 'Fm7'
 'G' 'G#' 'G#7' 'G#m' 'G#m7' 'G7' 'Gm' 'Gm7']
['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F

In [101]:
i = 1

tv_matrix = tfidf_model[i].toarray()

vocab = tfidf_vectorizer[i].get_feature_names_out()
tfidf_chords = pd.DataFrame(tv_matrix, columns = vocab)
tfidf_chords

,A,A7,Am,Am7,B,B7,Bb,Bb7,Bbm,Bbm7,Bm,Bm7,C,C#,C#7,C#m,C#m7,C7,Cm,Cm7,D,D7,Dm,Dm7,E,E7,Eb,Eb7,Ebm,Ebm7,Em,Em7,F,F#,F#7,F#m,F#m7,F7,Fm,Fm7,G,G#,G#7,G#m,G#m7,G7,Gm,Gm7
0,0.096717,0.000000,0.000000,0.045210,0.0,0.290675,0.000000,0.000000,0.0,0.095548,0.0,0.061840,0.151813,0.094245,0.000000,0.00000,0.0,0.205434,0.000000,0.000000,0.0,0.037845,0.000000,0.071119,0.437645,0.088860,0.000000,0.462625,0.000000,0.0,0.000000,0.000000,0.159336,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.517709,0.065885,0.0,0.000000,0.207987,0.0,0.236796
1,0.000000,0.108299,0.000000,0.133066,0.0,0.000000,0.000000,0.000000,0.0,0.281223,0.0,0.182013,0.000000,0.554777,0.000000,0.00000,0.0,0.000000,0.000000,0.259732,0.0,0.000000,0.000000,0.104661,0.000000,0.130770,0.253959,0.000000,0.000000,0.0,0.000000,0.137777,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.609573,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,0.000000,0.000000,0.000000,0.262650,0.0,0.168868,0.256757,0.000000,0.0,0.000000,0.0,0.359263,0.000000,0.547519,0.250245,0.00000,0.0,0.000000,0.000000,0.256333,0.0,0.000000,0.000000,0.000000,0.127125,0.322647,0.000000,0.000000,0.000000,0.0,0.000000,0.067987,0.000000,0.0,0.0,0.000000,0.190105,0.164711,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.302067,0.000000,0.0,0.000000
3,0.000000,0.000000,0.000000,0.524170,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.547596,0.000000,0.000000,0.00000,0.0,0.052929,0.000000,0.000000,0.0,0.048753,0.000000,0.412280,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.051315,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.497592,0.0,0.000000
4,0.000000,0.121667,0.000000,0.000000,0.0,0.000000,0.487119,0.179672,0.0,0.000000,0.0,0.000000,0.033465,0.000000,0.000000,0.00000,0.0,0.498137,0.000000,0.000000,0.0,0.041712,0.000000,0.000000,0.000000,0.000000,0.095102,0.000000,0.479610,0.0,0.000000,0.000000,0.043904,0.0,0.0,0.278018,0.000000,0.187494,0.000000,0.000000,0.0,0.000000,0.290469,0.0,0.000000,0.163742,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,0.000000,0.000000,0.000000,0.000000,0.0,0.208837,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.436283,0.000000,0.000000,0.64808,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.255479,0.157214,0.000000,0.000000,0.000000,0.130264,0.0,0.169934,0.000000,0.071547,0.0,0.0,0.151021,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.426940,0.0,0.000000
1808,0.000000,0.322223,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.199416,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.082853,0.000000,0.544947,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.307445,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.432721,0.0,0.000000,0.520388,0.0,0.000000
1809,0.000000,0.262272,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.324628,0.000000,0.000000,0.00000,0.0,0.000000,0.339845,0.000000,0.0,0.134875,0.704728,0.190096,0.000000,0.237516,0.000000,0.000000,0.000000,0.0,0.000000,0.166829,0.070982,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.264730,0.0,0.000000
1810,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.734458,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.262790,0.0,0.0,0.000000,0.000000,0.498778,0.000000,0.000000,0.0,0.189744,0.000000,0.0,0.000000,0.326695,0.0,0.000000


The `tfidf_model` is a list of the 4 models fitted with ngram size 1...4. 
Each of the model contains a sparse matrix with tunes being in the rows and the vocabulary being in the columns.

In [102]:
tfidf_model[1][100]

<1x48 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [103]:
tfidf_model

{1: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21387 stored elements in Compressed Sparse Row format>,
 2: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21387 stored elements in Compressed Sparse Row format>,
 3: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21387 stored elements in Compressed Sparse Row format>,
 4: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21378 stored elements in Compressed Sparse Row format>}

## Test for single tunes



In [104]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [111]:
model1 = 1
model2 = 2
model3 = 3
model4 = 4

df_sim = pd.DataFrame(columns=['reference_title',
                               'similar_title', 
                               f'sim{model1}', 
                               f'sim{model2}',
                               f'sim{model3}',
                               f'sim{model4}',
                               ])

for tune in tunes_eval_list:
  print(tune)
  s1 = title_to_index[tune]
  for s2 in range(len(data_tfidf)):
    if s2 != title_to_index[tune]:
      sim1 = cosine_similarity(tfidf_model[model1][s1], tfidf_model[model1][s2])[0][0]
      sim2 = cosine_similarity(tfidf_model[model2][s1], tfidf_model[model2][s2])[0][0]
      sim3 = cosine_similarity(tfidf_model[model3][s1], tfidf_model[model3][s2])[0][0]
      sim4 = cosine_similarity(tfidf_model[model4][s1], tfidf_model[model4][s2])[0][0]
      if sim1 > 0.8:
        df_sim.loc[len(df_sim)] = [tune,
                                  index_to_title[s2], 
                                  sim1, 
                                  sim2,
                                  sim3,
                                  sim4,
                                  ]

df_sim = df_sim.sort_values(f'sim{model2}', ascending=False)

Sweet Sue, Just You
These Foolish Things
Blue Moon
All Of Me
All God's Chillun Got Rhythm
I Got Rhythm
Bye Bye Blackbird
Old Fashioned Love
Dinah
Honeysuckle Rose
Misty


In [113]:
from google.colab import files

file = 'tfidf_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [112]:
df_sim.head(50)

,reference_title,similar_title,sim1,sim2,sim3,sim4
437,I Got Rhythm,I Got Rhythm,1.000000,1.000000,1.000000,1.000000
683,Honeysuckle Rose,Honeysuckle Rose,1.000000,1.000000,1.000000,1.000000
391,All God's Chillun Got Rhythm,They Didn't Believe Me,0.976789,0.976822,0.976149,0.976037
399,All God's Chillun Got Rhythm,Too Young To Go Steady,0.965435,0.962666,0.961631,0.957833
581,Dinah,I Like The Likes Of You,0.960769,0.959020,0.956768,0.954729
684,Honeysuckle Rose,How Are Things In Glocca Morra,0.964530,0.958438,0.958275,0.960798
552,Dinah,52nd Street Theme,0.962552,0.958296,0.957634,0.956288
280,All God's Chillun Got Rhythm,But Beautiful,0.961300,0.955523,0.954678,0.954618
345,All God's Chillun Got Rhythm,Little Willie Leaps,0.951911,0.950764,0.956084,0.956763
727,Honeysuckle Rose,Secret Love,0.946618,0.948079,0.945731,0.943973


## Test of the Contrafacts tunes

In [33]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  for ngram_size, tfidf_m in tfidf_model.items():
    row.append(cosine_similarity(tfidf_m[tune1], tfidf_m[tune2])[0][0])
  
  df_eval_tfidf.loc[len(df_eval_tfidf)] = row

df_eval_tfidf

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4
0,26-2,Confirmation,0.500697,0.492477,0.494181,0.501448
1,52nd Street Theme,I Got Rhythm,0.831848,0.825161,0.817133,0.809436
2,Ablution,All The Things You Are,0.917601,0.914305,0.916288,0.925313
3,Anthropology,I Got Rhythm,0.715753,0.706849,0.709068,0.703962
4,Ballade,As Long As I Live,0.638264,0.634892,0.623167,0.612081
5,Bright Mississippi,Sweet Georgia Brown,0.898530,0.891729,0.881984,0.870316
6,C.T.A.,I Got Rhythm,0.543100,0.535901,0.536527,0.539971
7,Celia,I Got Rhythm,0.362829,0.343311,0.344947,0.337100
8,Cottontail,I Got Rhythm,0.926148,0.923451,0.920177,0.916899
9,Countdown,Tune Up,0.674910,0.640808,0.639790,0.627937


In [34]:
print(tfidf_model[2][1171].toarray())

[[0.         0.21676466 0.         0.46608648 0.         0.0856189
  0.         0.08002741 0.         0.         0.         0.
  0.29065988 0.         0.         0.         0.         0.18153323
  0.         0.         0.         0.22294609 0.         0.5237075
  0.         0.         0.         0.         0.         0.
  0.         0.20682314 0.         0.         0.         0.
  0.09638605 0.         0.09923582 0.         0.         0.
  0.         0.         0.         0.45947178 0.         0.0871858 ]]


In [35]:
m = tfidf_model[1]
tfidf_cos = cosine_similarity(m)

In [38]:
from operator import itemgetter

tfidf_dict = {}
for row_index, row in enumerate(tfidf_cos):
  indices, L_sorted = zip(*sorted(enumerate(row), key=itemgetter(1), reverse=True))
  
  # do not report self-similiarity, therefore remove the reference tune
  remove_index = indices.index(row_index)
  
  indices = list(indices)
  indices.pop(remove_index)
  L_sorted = list(L_sorted)
  L_sorted.pop(remove_index)

  tfidf_dict[row_index] = {}
  tfidf_dict[row_index]['tune_indices'] = indices[:10]
  tfidf_dict[row_index]['score'] = [round(num, 3) for num in L_sorted[:10]] 



In [37]:
tfidf_file = 'tfidf_cossim.pickle'
with open(tfidf_file, 'wb') as f:
      # Pickle the 'data' dictionary using the highest protocol available.
      pickle.dump(tfidf_dict, f, pickle.HIGHEST_PROTOCOL)

from google.colab import files
files.download(tfidf_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Doc2Vec Embeddings

## Read and Tag Data

In [39]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [40]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [41]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [45]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 #4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 1, 
                                 2, 
                                 #3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=20, 
    min_count=2,
    #sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}, sample={sample}")
                        print(f"Corpus: {train_corpus[0]}")
                        model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                              dm = dm,
                                                              dm_concat = concat,
                                                              sample = sample,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-18 20:03:46,122 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n10,w5,mc2,s0.1,t2)', 'datetime': '2021-11-18T20:03:46.122271', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-18 20:03:46,130 : INFO : collecting all words and their counts
2021-11-18 20:03:46,132 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-18 20:03:46,151 : INFO : collected 48 word types and 1812 unique tags from a corpus of 1812 examples and 70731 words
2021-11-18 20:03:46,155 : INFO : Creating a fresh vocabulary
2021-11-18 20:03:46,158 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 48 unique words (100.0%% of original 48, drops 0)', 'datetime': '2021-11-18T20:03:46.158060', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubun

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=2, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-18 20:03:46,426 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:03:46,459 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:03:46,461 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 280039 effective words/s
2021-11-18 20:03:46,694 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:03:46,721 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:03:46,722 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.3s, 288573 effective words/s
2021-11-18 20:03:46,947 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:03:46,977 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:03:46,978 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.2s, 296125 effective words/s
2021-11-18 20:03:47,205 : INFO : worker thread finished

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=3, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-18 20:03:51,726 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:03:51,760 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:03:51,762 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 274923 effective words/s
2021-11-18 20:03:51,990 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:03:52,019 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:03:52,023 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.3s, 288102 effective words/s
2021-11-18 20:03:52,233 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:03:52,269 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:03:52,271 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.2s, 300058 effective words/s
2021-11-18 20:03:52,481 : INFO : worker thread finished

...

2021-11-18 20:04:39,305 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:39,328 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:39,329 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.2s, 456124 effective words/s
2021-11-18 20:04:39,460 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:39,483 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:39,485 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.1s, 493617 effective words/s
2021-11-18 20:04:39,616 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:39,638 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:39,640 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.1s, 500449 effective words/s
2021-11-18 20:04:39,782 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=2, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-18 20:04:42,751 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:42,774 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:42,776 : INFO : EPOCH - 1 : training on 68919 raw words (70411 effective words) took 0.2s, 389773 effective words/s
2021-11-18 20:04:42,937 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:42,966 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:42,968 : INFO : EPOCH - 2 : training on 68919 raw words (70411 effective words) took 0.2s, 380993 effective words/s
2021-11-18 20:04:43,115 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:43,142 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:43,144 : INFO : EPOCH - 3 : training on 68919 raw words (70411 effective words) took 0.2s, 423307 effective words/s
2021-11-18 20:04:43,300 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=3, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-18 20:04:46,679 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:46,711 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:46,716 : INFO : EPOCH - 1 : training on 68919 raw words (70411 effective words) took 0.2s, 363794 effective words/s
2021-11-18 20:04:46,873 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:46,903 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:46,904 : INFO : EPOCH - 2 : training on 68919 raw words (70411 effective words) took 0.2s, 387709 effective words/s
2021-11-18 20:04:47,064 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:47,092 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:47,094 : INFO : EPOCH - 3 : training on 68919 raw words (70411 effective words) took 0.2s, 393796 effective words/s
2021-11-18 20:04:47,262 : INFO : worker thread finished

### Trial with Models according to the Original Paper 

In [46]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

train_corpus = list(read_corpus(path_to_file, ngram_size=1))

common_kwargs = dict(
    vector_size=100, 
    epochs=50, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=5, 
    hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(train_corpus)
    model.train(train_corpus, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

2021-11-18 20:04:50,424 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,t2)', 'datetime': '2021-11-18T20:04:50.424761', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-18 20:04:50,427 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w10,mc2,t2)', 'datetime': '2021-11-18T20:04:50.427547', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-18 20:04:50,430 : INFO : using concatenative 1100-dimensional layer1
2021-11-18 20:04:50,433 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'datetime': '2021-11-18T20:04:50.433592', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event

Doc2Vec(dbow,d100,n5,mc2,t2) vocabulary scanned & state initialized


2021-11-18 20:04:57,559 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:57,589 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:57,591 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.2s, 295742 effective words/s
2021-11-18 20:04:57,806 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:57,835 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:57,836 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.2s, 311832 effective words/s
2021-11-18 20:04:58,032 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:04:58,061 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:04:58,063 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.2s, 333237 effective words/s
2021-11-18 20:04:58,270 : INFO : worker thread finished

Doc2Vec(dm/m,d100,n5,w10,mc2,t2) vocabulary scanned & state initialized


2021-11-18 20:05:09,854 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:05:09,885 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:05:09,887 : INFO : EPOCH - 1 : training on 70731 raw words (72543 effective words) took 0.3s, 283881 effective words/s
2021-11-18 20:05:10,095 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:05:10,127 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:05:10,128 : INFO : EPOCH - 2 : training on 70731 raw words (72543 effective words) took 0.2s, 312335 effective words/s
2021-11-18 20:05:10,345 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-18 20:05:10,380 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-18 20:05:10,382 : INFO : EPOCH - 3 : training on 70731 raw words (72543 effective words) took 0.2s, 304666 effective words/s
2021-11-18 20:05:10,612 : INFO : worker thread finished

Doc2Vec(dm/c,d100,n5,w5,mc2,t2) vocabulary scanned & state initialized


In [47]:
!pip install testfixtures

     |████████████████████████████████| 95 kB 2.6 MB/s 


In [48]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

2021-11-18 20:05:24,481 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-11-18 20:05:24,483 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-11-18 20:05:24,489 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2021-11-18T20:05:24.489679', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


In [49]:
models_by_name

OrderedDict([('Doc2Vec(dbow,d100,n5,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f0f6ffe2210>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f0f6ffe2650>),
             ('Doc2Vec(dm/c,d100,n5,w5,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f0f6fcae110>),
             ('dbow+dmm',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x7f0f6d6452d0>),
             ('dbow+dmc',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x7f0f6d645590>)])

In [50]:
from scipy import spatial

methods = ['Doc2Vec(dbow,d100,n5,mc2,t2)', 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'dbow+dmm', 'dbow+dmc']

# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'score_dbow', 'score_concat', 'score_dbow+dmm', 'score_dbow+dmc']

df_eval = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
    row = [index_to_title[tune1], index_to_title[tune2]]

    for m in methods:
        vec1 = models_by_name[m][tune1]
        vec2 = models_by_name[m][tune2]
      
        row.append(1 - spatial.distance.cosine(vec1, vec2))
  
    df_eval.loc[len(df_eval)] = row

df_eval

,Tune1,Tune2,score_dbow,score_concat,score_dbow+dmm,score_dbow+dmc
0,26-2,Confirmation,-0.032406,-0.023709,-0.210069,-0.014846
1,52nd Street Theme,I Got Rhythm,-0.078393,0.088038,0.053295,0.059435
2,Ablution,All The Things You Are,0.062243,0.138223,0.068353,0.085541
3,Anthropology,I Got Rhythm,0.559742,0.470191,0.343127,0.519262
4,Ballade,As Long As I Live,0.470839,0.030653,0.586932,0.309980
5,Bright Mississippi,Sweet Georgia Brown,0.791240,0.667455,0.307383,0.736960
6,C.T.A.,I Got Rhythm,0.465646,0.165473,0.191794,0.337829
7,Celia,I Got Rhythm,0.218067,-0.111592,0.259101,0.077163
8,Cottontail,I Got Rhythm,0.841551,0.620706,0.782753,0.744453
9,Countdown,Tune Up,0.632056,0.717612,0.606540,0.662698


### End of Other Trial

In [51]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

2021-11-18 20:05:24,960 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model
2021-11-18 20:05:24,974 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model.dv.* with mmap=None
2021-11-18 20:05:24,977 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model.wv.* with mmap=None
2021-11-18 20:05:24,980 : INFO : setting ignored attribute cum_table to None
2021-11-18 20:05:25,002 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_2_2_0.1.model', 'datetime': '2021-11-18T20:05:25.002827', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-18 20:05:25,005 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model
2021-11-18 20:05:25,011 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model.dv.* with

['autos_d2v_concat_2_2_0.1.model', 'autos_d2v_pv-dbow_2_3_0.1.model', 'autos_d2v_pv-dbow_2_2_0.1.model', 'autos_d2v_concat_1_3_0.1.model', 'autos_d2v_pv-dm_1_3_0.1.model', 'autos_d2v_concat_1_2_0.1.model', 'autos_d2v_pv-dbow_1_3_0.1.model', 'autos_d2v_pv-dbow_1_2_0.1.model', 'autos_d2v_pv-dm_2_2_0.1.model', 'autos_d2v_pv-dm_2_3_0.1.model', 'autos_d2v_concat_2_3_0.1.model', 'autos_d2v_pv-dm_1_2_0.1.model']
./models_doc2vec/autos_d2v_concat_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model
./models_doc2vec/autos_d2v_concat_1_3_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model
./models_doc2vec/autos_d2v_concat_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model


2021-11-18 20:05:25,159 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model.wv.* with mmap=None
2021-11-18 20:05:25,162 : INFO : setting ignored attribute cum_table to None
2021-11-18 20:05:25,165 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model', 'datetime': '2021-11-18T20:05:25.165880', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-18 20:05:25,168 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model
2021-11-18 20:05:25,174 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model.dv.* with mmap=None
2021-11-18 20:05:25,178 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model.wv.* with mmap=None
2021-11-18 20:05:25,180 : INFO : setting ignored attribute cum_table to None
2021-11-18 20:05:2

./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model
./models_doc2vec/autos_d2v_concat_2_3_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model


2021-11-18 20:05:25,376 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model.wv.* with mmap=None
2021-11-18 20:05:25,379 : INFO : setting ignored attribute cum_table to None
2021-11-18 20:05:25,385 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model', 'datetime': '2021-11-18T20:05:25.385635', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


In [52]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_concat_2_3_0.1.train', 'autos_d2v_pv-dbow_1_3_0.1.train', 'autos_d2v_concat_1_2_0.1.train', 'autos_d2v_pv-dm_2_2_0.1.train', 'autos_d2v_pv-dm_1_2_0.1.train', 'autos_d2v_pv-dm_2_3_0.1.train', 'autos_d2v_pv-dbow_2_3_0.1.train', 'autos_d2v_concat_2_2_0.1.train', 'autos_d2v_concat_1_3_0.1.train', 'autos_d2v_pv-dbow_2_2_0.1.train', 'autos_d2v_pv-dbow_1_2_0.1.train', 'autos_d2v_pv-dm_1_3_0.1.train']
./traindata_doc2vec/autos_d2v_concat_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_3_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_2_3_0.1.train
./traindata_doc2vec/autos_d2v_concat_2_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_2_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_2_0.1.train
./traindata_doc2vec/autos_d2v

In [53]:
for name, m in models.items():
  print(m)

Doc2Vec(dm/c,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dbow,d100,n10,mc2,s0.1,t2)
Doc2Vec(dbow,d100,n10,mc2,s0.1,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dbow,d100,n10,mc2,s0.1,t2)
Doc2Vec(dbow,d100,n10,mc2,s0.1,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,s0.1,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,s0.1,t2)


In [54]:
def compare_doc_models(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [55]:
df_score = compare_doc_models(models)


*** Run 0
autos_d2v_concat_2_2_0.1.model
[TaggedDocument(words=['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], tags=[0]), TaggedDocument(words=['Am7-Cm7', 'Cm7-Eb', 'Eb-Em7', 'Em7-A7', 'A7-Dm7', 'Dm7-Bm7', 'Bm7-Bbm7', 'Bbm7-Fm7', 'Fm7-E7', 'E7-Fm7', 'Fm7-C#', 'C#-Fm7', 'Fm7-C#'], tags=[1]), TaggedDocument(words=['Am7-C#', 'C#-Bm7', 'Bm7-E7', 'E7-Am7', 'Am7-C#', 'C#-Bm7', 'Bm7-E7', 'E7-Cm7', 'Cm7-F7', 'F7-Bb', 'Bb-G#m7', 'G#m7-C#7', 'C#7-F#m7', 'F#m7-B7', 'B7-E', 'E-E7', 'E7-Am7', 'Am7-C#', 'C#-Bm7', 'Bm7-E7', 'E7-Am7', '

In [56]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,19.315673
autos_d2v_concat_1_3_0.1.model,18.708609
autos_d2v_concat_2_2_0.1.model,95.143488
autos_d2v_concat_2_3_0.1.model,94.922737
autos_d2v_pv-dbow_1_2_0.1.model,4.415011
autos_d2v_pv-dbow_1_3_0.1.model,4.249448
autos_d2v_pv-dbow_2_2_0.1.model,4.249448
autos_d2v_pv-dbow_2_3_0.1.model,3.752759
autos_d2v_pv-dm_1_2_0.1.model,33.498896


In [57]:
def compare_doc_models2(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    topn = 5

    names = []
    scores = []
    runs = []
    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        tune_sim = 0

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=topn)
            if doc_id in [key for key, value in sims]:
              tune_sim += 1

        tunes_not_sim = len(train_corpus) - tune_sim
        score = 100 * tunes_not_sim/(tune_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in top {topn} place: {tune_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [58]:
df_score2 = compare_doc_models2(models)


*** Run 0
autos_d2v_concat_2_2_0.1.model
[TaggedDocument(words=['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], tags=[0]), TaggedDocument(words=['Am7-Cm7', 'Cm7-Eb', 'Eb-Em7', 'Em7-A7', 'A7-Dm7', 'Dm7-Bm7', 'Bm7-Bbm7', 'Bbm7-Fm7', 'Fm7-E7', 'E7-Fm7', 'Fm7-C#', 'C#-Fm7', 'Fm7-C#'], tags=[1]), TaggedDocument(words=['Am7-C#', 'C#-Bm7', 'Bm7-E7', 'E7-Am7', 'Am7-C#', 'C#-Bm7', 'Bm7-E7', 'E7-Cm7', 'Cm7-F7', 'F7-Bb', 'Bb-G#m7', 'G#m7-C#7', 'C#7-F#m7', 'F#m7-B7', 'B7-E', 'E-E7', 'E7-Am7', 'Am7-C#', 'C#-Bm7', 'Bm7-E7', 'E7-Am7', '

In [59]:
pd.pivot_table(df_score2, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,8.278146
autos_d2v_concat_1_3_0.1.model,7.505519
autos_d2v_concat_2_2_0.1.model,92.715232
autos_d2v_concat_2_3_0.1.model,92.494481
autos_d2v_pv-dbow_1_2_0.1.model,0.165563
autos_d2v_pv-dbow_1_3_0.1.model,0.220751
autos_d2v_pv-dbow_2_2_0.1.model,0.220751
autos_d2v_pv-dbow_2_3_0.1.model,0.331126
autos_d2v_pv-dm_1_2_0.1.model,15.121413


In [60]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

20211118_201020


In [61]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [62]:
 !zip -r models.zip models_doc2vec/ 

  adding: models_doc2vec/ (stored 0%)
  adding: models_doc2vec/autos_d2v_concat_2_2_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model (deflated 9%)
  adding: models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model (deflated 9%)
  adding: models_doc2vec/autos_d2v_concat_1_3_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_concat_1_2_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model (deflated 9%)
  adding: models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model (deflated 8%)
  adding: models_doc2vec/autos_d2v_concat_2_3_0.1.model (deflated 7%)
  adding: models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model (deflated 7%)


In [63]:
from google.colab import files
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
files.download(score_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Assess the best Model

In [114]:
#best_model = 'autos_d2v_pv-dbow_3_4_0.1'
best_model = 'autos_d2v_concat_1_3_0.1'

In [115]:
model = models[f"{best_model}.model"]

### Visualize Chord and Tune Vectors

In [144]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, 
            random_state=0,
            perplexity = 20,
            learning_rate=100.0,
            #n_iter_without_progress=200,
            n_iter=5000
            )
#np.set_printoptions(suppress=True)
T = tsne.fit_transform(model.wv.vectors)

projected = pd.DataFrame(T)

fig = px.scatter(
    projected, 
    x=0, y=1,
    #color='mode',
    text=model.wv.index_to_key,
    width=600, height=500,
    title="T-SNE applied to Chord Vectors for Tunes in Cmaj/Amin"
)
fig.update_traces(textposition='top center')
fig.update_traces(textfont_size=12, selector=dict(type='scatter'))
fig.show()

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



In [161]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, 
            random_state=42,
            perplexity = 10,
            learning_rate = 'auto',
            init='pca',
            )
T = tsne.fit_transform(model.dv.vectors)
projected = pd.DataFrame(T)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning:

The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.



In [162]:
fig = px.scatter(
    projected, 
    x=0, y=1,
    #color='mode',
    hover_name=titles['title'],
    width=600, height=500,
    title="T-SNE applied to Document Vectors for Tunes in Cmaj/Amin"
)
fig.update_traces(textposition='top center')
fig.update_traces(textfont_size=12, selector=dict(type='scatter'))
fig.show()

#### Assess self-similarity

In [67]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [68]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

Counter({0: 1471, 1: 125, 2: 45, 3: 27, 5: 17, 4: 15, 6: 11, 7: 10, 9: 6, 12: 6, 11: 6, 10: 6, 16: 5, 8: 4, 17: 4, 19: 3, 14: 3, 15: 2, 18: 2, 33: 2, 21: 2, 71: 2, 29: 2, 323: 1, 64: 1, 42: 1, 37: 1, 100: 1, 41: 1, 272: 1, 82: 1, 66: 1, 92: 1, 30: 1, 46: 1, 84: 1, 36: 1, 47: 1, 32: 1, 238: 1, 154: 1, 65: 1, 62: 1, 45: 1, 215: 1, 162: 1, 304: 1, 120: 1, 49: 1, 111: 1, 103: 1, 38: 1, 83: 1, 56: 1, 35: 1, 51: 1, 27: 1, 391: 1, 28: 1})

Tunes similar to themselves in first place: 1471
Tunes not similar in first place: 341
% of tunes not similar to themselves in first place: 18.82%


In [69]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

Tunes that are similar to other tunes before they are similar to themselves:
'A Foggy Day': 1
'Ac-Cent-Tchu-Ate The Positive': 1
'After You': 15
'Alexander's Ragtime Band': 6
'All God's Chillun Got Rhythm': 5
'All The Things You Are': 1
'All Through The Night': 1
'All Too Soon': 3
'Am I Blue': 2
'Among My Souvenirs': 4
'Anthropology': 3
'At Last': 9
'Au Privave': 4
'Avalon': 2
'Bags and Trane': 2
'Barbados': 3
'Barbara': 4
'Basin Street Blues': 1
'Bein' Green': 5
'Bessie's Blues': 6
'Best Thing For You Is Me, The': 1
'Best Things In Life Are Free, The': 15
'Better Than Anything': 2
'Between The Devil And The Deep Blue Sea': 2
'Bill Bailey': 323
'Billie's Bounce': 3
'Billy Boy': 2
'Birth Of The Blues, The': 1
'Blue Monk': 18
'Blue Moon': 5
'Blue Room, The': 5
'Blue Skies': 1
'Blue Sphere': 16
'Bluehawk': 8
'Blues For Alice': 12
'Blues In The Closet': 2
'Bluesette': 2
'Bourbon Street Parade': 9
'Brazil (Aquarela Do Brasil)': 1
'Bright Boy': 64
'Bright Mississippi': 1
'Broadway': 1
'But B

#### Determine Similarity of Tunes

In [70]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [71]:
tunes_eval_list = [
                   'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [72]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')



'Sweet Sue, Just You' is most similar to:
	0.861: Moten Swing
	0.861: Rosalie
	0.853: Sweet Sue
	0.783: I've Got My Love To Keep Me Warm
	0.760: Sno' Peas
	0.759: Three Little Words
	0.741: Marie
	0.730: Road Song
	0.730: Roses Of Picardy
	0.728: Alone Too Long

'These Foolish Things' is most similar to:
	0.763: Bohemia After Dark
	0.731: S.O.S.
	0.719: Touch Of Your Lips, The
	0.703: On The Sunny Side Of The Street
	0.681: Catch Me
	0.676: Waltz For Debby
	0.676: Summertime
	0.674: Smiles (That Make Us Happy)
	0.672: Cottontail
	0.656: Witch Hunt

'Blue Moon' is most similar to:
	0.738: Night Has A Thousand Eyes, The
	0.731: I May Be Wrong
	0.707: There Will Never Be Another You
	0.673: Night Dreamer
	0.662: My Heart Stood Still
	0.644: Countdown
	0.640: S'posin'
	0.638: But Not For Me
	0.629: Lily of The Valley
	0.626: Making Whoopee

'Hundred Years From Today, A' is most similar to:
	0.701: Chicken
	0.668: My Song
	0.658: A Taste Of Honey
	0.641: 'S Wonderful
	0.626: Love Walked In

#### Evaluate Similarity for Jazz Contrafacts



In [73]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

,Tune1,Tune2,doc2vec
0,26-2,Confirmation,-0.005596
1,52nd Street Theme,I Got Rhythm,0.595095
2,Ablution,All The Things You Are,0.894314
3,Anthropology,I Got Rhythm,0.187682
4,Ballade,As Long As I Live,0.086513
5,Bright Mississippi,Sweet Georgia Brown,0.671953
6,C.T.A.,I Got Rhythm,0.003111
7,Celia,I Got Rhythm,-0.224643
8,Cottontail,I Got Rhythm,0.741344
9,Countdown,Tune Up,0.572571


##### Compare results from TF-IDF and doc2vec



In [74]:
df_eval_tfidf.merge(df_eval_doc2vec)

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4,doc2vec
0,26-2,Confirmation,0.500697,0.492477,0.494181,0.501448,-0.005596
1,52nd Street Theme,I Got Rhythm,0.831848,0.825161,0.817133,0.809436,0.595095
2,Ablution,All The Things You Are,0.917601,0.914305,0.916288,0.925313,0.894314
3,Anthropology,I Got Rhythm,0.715753,0.706849,0.709068,0.703962,0.187682
4,Ballade,As Long As I Live,0.638264,0.634892,0.623167,0.612081,0.086513
5,Bright Mississippi,Sweet Georgia Brown,0.898530,0.891729,0.881984,0.870316,0.671953
6,C.T.A.,I Got Rhythm,0.543100,0.535901,0.536527,0.539971,0.003111
7,Celia,I Got Rhythm,0.362829,0.343311,0.344947,0.337100,-0.224643
8,Cottontail,I Got Rhythm,0.926148,0.923451,0.920177,0.916899,0.741344
9,Countdown,Tune Up,0.674910,0.640808,0.639790,0.627937,0.572571


##### Verbose Testing Output

In [75]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': -0.005596276372671127
'26-2' is most similar to:
	0.732: Drop Me Off In Harlem
	0.690: Estate
	0.679: Whisper Not
	0.676: Bossa Antigua
	0.672: Perhaps

'52nd Street Theme' to 'I Got Rhythm': 0.5950954556465149
'52nd Street Theme' is most similar to:
	0.889: I've Got My Fingers Crossed
	0.868: Shine On Your Shoes, A
	0.849: When The Moon Comes Over The Mountain
	0.845: Why Do I Love You
	0.845: Misty

'Ablution' to 'All The Things You Are': 0.8943141102790833
'Ablution' is most similar to:
	0.894: All The Things You Are
	0.763: Mc Jolt
	0.743: Zingaro (Retrato Em Branco E Preto)
	0.727: Irresistable You
	0.726: Days Of Wine And Roses

'Anthropology' to 'I Got Rhythm': 0.1876823902130127
'Anthropology' is most similar to:
	0.975: Oleo
	0.971: Salt Peanuts
	0.969: Shaw 'Nuff
	0.961: Moose The Mooche
	0.961: No Moe

'Ballade' to 'As Long As I Live': 0.08651280403137207
'Ballade' is most similar to:
	0.829: Agua De Beber
	0.790: Taking A Chance On Love
	0.778: Fo

In [76]:
model.dv.similarity(0, 1)

0.5249816

#### Download Model

In [ ]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")